**DWT features Extraction**

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pywt
from scipy.stats import skew, kurtosis
from skimage import color, io
from skimage.measure import shannon_entropy
from tqdm import tqdm

# Path to your dataset folder
dataset_folder = r"D:\new dataset\KTH_TIPS"

# Function to preprocess the image
def preprocess_image(image_path, target_size=(128, 128)):
    # Load the image
    img = io.imread(image_path)
    if img.shape[-1] == 4:  # RGBA to RGB
        img = img[:, :, :3]
    resized_img = cv2.resize(img, target_size)  # Resize to target size
    normalized_img = resized_img / 255.0  # Normalize to [0, 1]
    return normalized_img

# Function to calculate features from wavelet sub-bands
def calculate_wavelet_features(image):
    # Perform 2D Discrete Wavelet Transform (DWT)
    coeffs_dwt = pywt.dwt2(image, 'haar')
    LL, (LH, HL, HH) = coeffs_dwt

    # Sub-band names and values
    subbands = {"LL": LL, "LH": LH, "HL": HL, "HH": HH}
    features = {}

    # Extract statistical features for each sub-band
    for band, subband in subbands.items():
        subband_flat = subband.flatten()  # Flatten for statistical calculations
        features.update({
            f"{band}_Mean": np.mean(subband),
            f"{band}_StdDev": np.std(subband),
            f"{band}_Variance": np.var(subband),
            f"{band}_Energy": np.sum(np.square(subband)),
            f"{band}_Entropy": shannon_entropy(subband),
            f"{band}_Skewness": skew(subband_flat),
            f"{band}_Kurtosis": kurtosis(subband_flat),
            f"{band}_Min": np.min(subband),
            f"{band}_Max": np.max(subband),
            f"{band}_Range": np.ptp(subband),  # Peak-to-peak (Max-Min)
        })
    return features

# Main feature extraction pipeline
feature_data = []

for class_name in os.listdir(dataset_folder):
    class_path = os.path.join(dataset_folder, class_name)
    if os.path.isdir(class_path):
        files = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg', '.jfif'))]
        with tqdm(total=len(files), desc=f"Processing {class_name}", unit="file") as pbar:
            for file_name in files:
                image_path = os.path.join(class_path, file_name)

                # Preprocess the image
                grayscale_image = preprocess_image(image_path)

                # Extract wavelet features
                wavelet_features = calculate_wavelet_features(grayscale_image)

                # Combine all features
                combined_features = {
                    "Class": class_name,
                    "Image Name": file_name,
                    **wavelet_features,
                }

                # Append features to the dataset
                feature_data.append(combined_features)
                pbar.update(1)

# Create a DataFrame and save it as a CSV file
df = pd.DataFrame(feature_data)
output_csv_path = r"D:\new dataset\new dataset\DWT\DWT_wavelet_features.csv"
df.to_csv(output_csv_path, index=False)
print("Feature extraction completed. Data saved to:", output_csv_path)


**GLCM features Extraction**

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from scipy.stats import skew, kurtosis
from skimage.measure import shannon_entropy

# Path to the dataset folders
dataset_path = r"D:\new dataset\KTH_TIPS"
categories = ["aluminium_foil", "brown_bread", "corduroy", "cotton", "cracker", "orange_peel","linen","sandpaper","sponge","styrofoam"]  # Subfolders for fresh and rotten fruits


# Function to extract features for a single image
def extract_features(image_path, patch_size=7):  # Increased patch size for better performance
    # Load and preprocess the image
    image = cv2.imread(image_path)
    # For only 3 channels (RGB)
    if image.shape[-1] == 4:
        image = image[:, :, :3]
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(gray_image, (128, 128))  # Reduced size for faster computation

    # Initialize feature maps
    contrast_map = np.zeros_like(resized_image, dtype=float)
    homogeneity_map = np.zeros_like(resized_image, dtype=float)
    energy_map = np.zeros_like(resized_image, dtype=float)
    correlation_map = np.zeros_like(resized_image, dtype=float)
    entropy_map = np.zeros_like(resized_image, dtype=float)

    # Calculate local GLCM features using a sliding window
    for i in range(0, resized_image.shape[0] - patch_size, patch_size):
        for j in range(0, resized_image.shape[1] - patch_size, patch_size):
            patch = resized_image[i:i + patch_size, j:j + patch_size]
            patch_glcm = graycomatrix(patch, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
            contrast_map[i:i + patch_size, j:j + patch_size] = graycoprops(patch_glcm, 'contrast')[0, 0]
            homogeneity_map[i:i + patch_size, j:i + patch_size] = graycoprops(patch_glcm, 'homogeneity')[0, 0]
            energy_map[i:i + patch_size, j:i + patch_size] = graycoprops(patch_glcm, 'energy')[0, 0]
            correlation_map[i:i + patch_size, j:i + patch_size] = graycoprops(patch_glcm, 'correlation')[0, 0]
            entropy_map[i:i + patch_size, j:i + patch_size] = -np.sum(patch_glcm * np.log2(patch_glcm + (patch_glcm == 0)))

    # Compute global averages of the feature maps
    features = {
        "contrast": np.mean(contrast_map),
        "homogeneity": np.mean(homogeneity_map),
        "energy": np.mean(energy_map),
        "correlation": np.mean(correlation_map),
        "entropy": np.mean(entropy_map),
        "mean": np.mean(resized_image),
        "std_deviation": np.std(resized_image),
        "variance": np.var(resized_image),
        "skewness": skew(resized_image.flatten()),
        "kurtosis": kurtosis(resized_image.flatten()),
        "min": np.min(resized_image),
        "max": np.max(resized_image),
        "range": np.ptp(resized_image)
    }
    return features

# Initialize a list to store data
data = []

# Progress tracking variables
total_images = sum(len(files) for _, _, files in os.walk(dataset_path) if files)
processed_images = 0

# Iterate through the dataset
for category in categories:
    category_path = os.path.join(dataset_path, category)
    label = category  # Label is the category name

    for filename in os.listdir(category_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Add other extensions if needed
            image_path = os.path.join(category_path, filename)
            features = extract_features(image_path)
            
            # Include filename and category in the features
            features["filename"] = filename  # Add the filename
            features["category"] = label  # Add the label (category)

            data.append(features)

            # Update progress
            processed_images += 1
            progress = (processed_images / total_images) * 100
            print(f"Progress: {progress:.2f}% images processed", end="\r")

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Display the first 5 rows of the DataFrame
print("\nFirst 5 rows of the DataFrame:")
print(df.head())

# Save the DataFrame to a CSV file
output_csv_path = r"D:\new dataset\new dataset\GLCM\glcm_features.csv"  # Replace with your desired CSV file path
df.to_csv(output_csv_path, index=False)

print(f"\nFeature extraction completed. Data saved to {output_csv_path}.")


**SWT Features Extraction**

In [ ]:
import os
import numpy as np
import pandas as pd
import pywt
from skimage import io, color
from skimage.measure import shannon_entropy
from tqdm import tqdm
import cv2  # For resizing

# Path to your dataset folder (update this path)
dataset_folder =r"D:\new dataset\KTH_TIPS"

# Function to preprocess the image
def preprocess_image(image_path, target_size=(128, 128)):
    # Load the image
    img = io.imread(image_path)
    
    # If the image has an alpha channel, remove it (convert RGBA to RGB)
    if img.ndim == 3 and img.shape[-1] == 4:
        img = img[:, :, :3]
    
    # Convert RGB to grayscale if the image is not already grayscale
    if img.ndim == 3:  # Check if the image has 3 channels
        grayscale_img = color.rgb2gray(img)
    else:
        grayscale_img = img  # The image is already grayscale
    
    # Resize the grayscale image to the target size
    resized_img = cv2.resize(grayscale_img, target_size)
    
    # Normalize pixel values to the range [0, 1]
    normalized_img = resized_img / 255.0
    
    return normalized_img

# Function to calculate features for each sub-band
def calculate_features(sub_band):
    features = {}
    # Calculate mean
    features['Mean'] = np.mean(sub_band)
    
    # Calculate standard deviation
    features['Std_Dev'] = np.std(sub_band)
    
    # Calculate variance
    features['Variance'] = np.var(sub_band)
    
    # Calculate energy (sum of squared values)
    features['Energy'] = np.sum(np.square(sub_band))
    
    # Calculate entropy using Shannon entropy
    features['Entropy'] = shannon_entropy(sub_band)
    
    # Calculate contrast (standard deviation as a simple measure)
    features['Contrast'] = np.std(sub_band)
    
    # Calculate homogeneity
    features['Homogeneity'] = 1 / (1 + features['Variance'])
    
    return features

# Function to calculate correlation between sub-bands
def calculate_correlation(sub_band1, sub_band2):
    return np.corrcoef(sub_band1.flatten(), sub_band2.flatten())[0, 1]

# Initialize a list to store feature data for the DataFrame
feature_data = []

# Loop through each class folder (e.g., apple, peach, etc.)
for class_name in os.listdir(dataset_folder):
    class_path = os.path.join(dataset_folder, class_name)
    
    # Process only directories
    if os.path.isdir(class_path):
        total_files = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg', '.jfif'))])
        
        with tqdm(total=total_files, desc=f"Processing {class_name} images", unit="file") as pbar:
            for file_name in os.listdir(class_path):
                if file_name.endswith(('.jpg', '.png', '.jpeg', '.jfif')):
                    image_path = os.path.join(class_path, file_name)
                    
                    # Preprocess the image
                    preprocessed_image = preprocess_image(image_path)
                    
                    # Perform 2D SWT (Stationary Wavelet Transform)
                    coeffs = pywt.swt2(preprocessed_image, 'haar', level=1)  # Single-level SWT
                    LL, (LH, HL, HH) = coeffs[0]  # Extract subbands from level 1
                    
                    # Calculate features for each sub-band
                    features_LL = calculate_features(LL)
                    features_LH = calculate_features(LH)
                    features_HL = calculate_features(HL)
                    features_HH = calculate_features(HH)
                    
                    # Calculate correlations between sub-bands
                    correlation_LL_LH = calculate_correlation(LL, LH)
                    correlation_LL_HL = calculate_correlation(LL, HL)
                    correlation_LL_HH = calculate_correlation(LL, HH)
                    correlation_LH_HL = calculate_correlation(LH, HL)
                    correlation_LH_HH = calculate_correlation(LH, HH)
                    correlation_HL_HH = calculate_correlation(HL, HH)
                    
                    # Combine features and add class label
                    feature_row = {
                        'Class': class_name,
                        'Image Name': file_name,
                        # Features for LL sub-band
                        'LL_Mean': features_LL['Mean'], 'LL_Std_Dev': features_LL['Std_Dev'], 
                        'LL_Variance': features_LL['Variance'], 'LL_Energy': features_LL['Energy'],
                        'LL_Entropy': features_LL['Entropy'], 'LL_Contrast': features_LL['Contrast'],
                        'LL_Homogeneity': features_LL['Homogeneity'],
                        
                        # Features for LH sub-band
                        'LH_Mean': features_LH['Mean'], 'LH_Std_Dev': features_LH['Std_Dev'], 
                        'LH_Variance': features_LH['Variance'], 'LH_Energy': features_LH['Energy'],
                        'LH_Entropy': features_LH['Entropy'], 'LH_Contrast': features_LH['Contrast'],
                        'LH_Homogeneity': features_LH['Homogeneity'],
                        
                        # Features for HL sub-band
                        'HL_Mean': features_HL['Mean'], 'HL_Std_Dev': features_HL['Std_Dev'], 
                        'HL_Variance': features_HL['Variance'], 'HL_Energy': features_HL['Energy'],
                        'HL_Entropy': features_HL['Entropy'], 'HL_Contrast': features_HL['Contrast'],
                        'HL_Homogeneity': features_HL['Homogeneity'],
                        
                        # Features for HH sub-band
                        'HH_Mean': features_HH['Mean'], 'HH_Std_Dev': features_HH['Std_Dev'], 
                        'HH_Variance': features_HH['Variance'], 'HH_Energy': features_HH['Energy'],
                        'HH_Entropy': features_HH['Entropy'], 'HH_Contrast': features_HH['Contrast'],
                        'HH_Homogeneity': features_HH['Homogeneity'],
                        
                        # Correlations between sub-bands
                        'LL_LH_Correlation': correlation_LL_LH, 'LL_HL_Correlation': correlation_LL_HL,
                        'LL_HH_Correlation': correlation_LL_HH, 'LH_HL_Correlation': correlation_LH_HL,
                        'LH_HH_Correlation': correlation_LH_HH, 'HL_HH_Correlation': correlation_HL_HH
                    }
                    feature_data.append(feature_row)
                    pbar.update(1)  # Update progress bar

# Create a DataFrame from the feature data
df = pd.DataFrame(feature_data)

# Save the DataFrame to a CSV file
output_csv_path = r"D:\new dataset\new dataset\swt\sWT_features_with_labels.csv"
df.to_csv(output_csv_path, index=False)

print("Feature extraction completed. Data saved to:", output_csv_path)


**SWT+GLCM FEATURES EXTRACTION**

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pywt
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy

# Path to the dataset folders
dataset_path = r"D:\new dataset\KTH_TIPS"
categories = ["aluminium_foil", "brown_bread", "corduroy", "cotton", "cracker", "orange_peel","linen","sandpaper","sponge","styrofoam"] 

# Function to preprocess the image
def preprocess_image(image_path, target_size=(128, 128)):
    image = cv2.imread(image_path)
    if image.shape[-1] == 4:  # If RGBA, convert to RGB
        image = image[:, :, :3]
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    resized_image = cv2.resize(gray_image, target_size)  # Resize to uniform size
    return resized_image

# Function to extract SWT and GLCM features
def extract_swt_glcm_features(image_path):
    # Preprocess the image
    preprocessed_image = preprocess_image(image_path)

    # Apply Stationary Wavelet Transform (SWT)
    coeffs_swt = pywt.swt2(preprocessed_image, wavelet='haar', level=1, norm=True)
    LL, (LH, HL, HH) = coeffs_swt[0]  # Extract the first-level subbands

    # Extract GLCM features from the LL subband
    glcm = graycomatrix(LL.astype(np.uint8), distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

    # Calculate GLCM properties
    glcm_features = {
        "contrast": np.mean(graycoprops(glcm, 'contrast')),
        "homogeneity": np.mean(graycoprops(glcm, 'homogeneity')),
        "energy": np.mean(graycoprops(glcm, 'energy')),
        "correlation": np.mean(graycoprops(glcm, 'correlation')),
        "dissimilarity": np.mean(graycoprops(glcm, 'dissimilarity')),
        "ASM": np.mean(graycoprops(glcm, 'ASM')),  # Angular Second Moment
        "entropy": shannon_entropy(LL)  # Entropy of the LL subband
    }
    return glcm_features

# Initialize a list to store the extracted features
data = []

# Progress tracking
total_images = sum(len(files) for _, _, files in os.walk(dataset_path) if files)
processed_images = 0

# Iterate through the dataset
for category in categories:
    category_path = os.path.join(dataset_path, category)
    label = category  # Use the folder name as the label

    for filename in os.listdir(category_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(category_path, filename)
            features = extract_swt_glcm_features(image_path)
            features["filename"] = filename
            features["category"] = label
            data.append(features)

            # Update progress
            processed_images += 1
            progress = (processed_images / total_images) * 100
            print(f"Progress: {progress:.2f}% images processed", end="\r")

# Convert the extracted features into a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
output_csv_path = r"D:\new dataset\new dataset\swt+glcm\swt_glcm_features.csv"
df.to_csv(output_csv_path, index=False)

print(f"\nFeature extraction completed. Data saved to {output_csv_path}.")


**DWT+GLCM Features Extraction**

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pywt
from scipy.stats import skew, kurtosis
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy

# Path to the dataset
dataset_path = r"D:\new dataset\KTH_TIPS"
categories = ["aluminium_foil", "brown_bread", "corduroy", "cotton", "cracker", "orange_peel","linen","sandpaper","sponge","styrofoam"] 

# Function to preprocess the image
def preprocess_image(image_path, target_size=(128, 128)):
    image = cv2.imread(image_path)
    if image.shape[-1] == 4:  # If RGBA, convert to RGB
        image = image[:, :, :3]
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    resized_image = cv2.resize(gray_image, target_size)  # Resize to uniform size
    return resized_image

# Function to apply DWT and extract features
def extract_dwt_glcm_features(image_path):
    # Preprocess the image
    preprocessed_image = preprocess_image(image_path)

    # Apply Discrete Wavelet Transform (DWT)
    coeffs_dwt = pywt.dwt2(preprocessed_image, wavelet='haar')
    LL, (LH, HL, HH) = coeffs_dwt  # Extract subbands

    # Extract GLCM features from the LL subband
    glcm = graycomatrix(LL.astype(np.uint8), distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=512, symmetric=True, normed=True)

    # Calculate GLCM properties
    glcm_features = {
        "Mean": np.mean(LL),
        "StdDev": np.std(LL),
        "Variance": np.var(LL),
        "contrast": np.mean(graycoprops(glcm, 'contrast')),
        "homogeneity": np.mean(graycoprops(glcm, 'homogeneity')),
        "energy": np.mean(graycoprops(glcm, 'energy')),
        "correlation": np.mean(graycoprops(glcm, 'correlation')),
        "dissimilarity": np.mean(graycoprops(glcm, 'dissimilarity')),
        "ASM": np.mean(graycoprops(glcm, 'ASM')),  # Angular Second Moment
        "entropy": shannon_entropy(LL)  # Entropy of the LL subband
    }
    return glcm_features

# Initialize a list to store the extracted features
data = []

# Progress tracking
total_images = sum(len(files) for _, _, files in os.walk(dataset_path) if files)
processed_images = 0

# Iterate through the dataset
for category in categories:
    category_path = os.path.join(dataset_path, category)
    label = category  # Use the folder name as the label

    for filename in os.listdir(category_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(category_path, filename)
            features = extract_dwt_glcm_features(image_path)
            features["filename"] = filename
            features["category"] = label
            data.append(features)

            # Update progress
            processed_images += 1
            progress = (processed_images / total_images) * 100
            print(f"Progress: {progress:.2f}% images processed", end="\r")

# Convert the extracted features into a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
output_csv_path = r"D:\new dataset\new dataset\dwt_glcm_features.csv"
df.to_csv(output_csv_path, index=False)

print(f"\nFeature extraction completed. Data saved to {output_csv_path}.")


**DWT+SWT+GLCM Features Extraction**

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pywt
from skimage import color, io
from skimage.measure import shannon_entropy
from skimage.feature import graycomatrix, graycoprops
from tqdm import tqdm

# Path to your dataset folder (update this path)
dataset_folder = r"D:\new dataset\KTH_TIPS"
# Function to preprocess the image
# Function to preprocess the image
def preprocess_image(image_path, target_size=(128, 128)):
    # Load the image
    img = io.imread(image_path)
    
    # If the image has an alpha channel, remove it (convert RGBA to RGB)
    if img.ndim == 3 and img.shape[-1] == 4:
        img = img[:, :, :3]
    
    # Convert RGB to grayscale if the image is not already grayscale
    if img.ndim == 3:  # Check if the image has 3 channels
        grayscale_img = color.rgb2gray(img)
    else:
        grayscale_img = img  # The image is already grayscale
    
    # Resize the grayscale image to the target size
    resized_img = cv2.resize(grayscale_img, target_size)
    
    # Normalize pixel values to the range [0, 1]
    normalized_img = resized_img / 255.0
    
    return normalized_img

# Function to calculate DWT/SWT features
def calculate_wavelet_features(sub_band):
    features = {
        "Mean": np.mean(sub_band),
        "Std_Dev": np.std(sub_band),
        "Variance": np.var(sub_band),
        "Energy": np.sum(np.square(sub_band)),
        "Entropy": shannon_entropy(sub_band),
        "Contrast": np.std(sub_band),
        "Homogeneity": 1 / (1 + np.var(sub_band))
    }
    return features

# Function to calculate correlation between wavelet sub-bands
def calculate_correlation(sub_band1, sub_band2):
    return np.corrcoef(sub_band1.flatten(), sub_band2.flatten())[0, 1]

# Function to extract GLCM features
def calculate_glcm_features(image, patch_size=7):
    # Resize and preprocess for GLCM
    resized_image = cv2.resize(image, (128, 128))
    
    # Scale the image to the range [0, 255] and convert to uint8
    resized_image = (resized_image * 255).astype(np.uint8)
    
    # Initialize GLCM feature maps
    contrast_map = np.zeros_like(resized_image, dtype=float)
    homogeneity_map = np.zeros_like(resized_image, dtype=float)
    energy_map = np.zeros_like(resized_image, dtype=float)
    correlation_map = np.zeros_like(resized_image, dtype=float)
    entropy_map = np.zeros_like(resized_image, dtype=float)

    # Calculate local GLCM features using a sliding window
    for i in range(0, resized_image.shape[0] - patch_size, patch_size):
        for j in range(0, resized_image.shape[1] - patch_size, patch_size):
            patch = resized_image[i:i + patch_size, j:j + patch_size]
            patch_glcm = graycomatrix(patch, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
            contrast_map[i:i + patch_size, j:j + patch_size] = graycoprops(patch_glcm, 'contrast')[0, 0]
            homogeneity_map[i:i + patch_size, j:j + patch_size] = graycoprops(patch_glcm, 'homogeneity')[0, 0]
            energy_map[i:i + patch_size, j:j + patch_size] = graycoprops(patch_glcm, 'energy')[0, 0]
            correlation_map[i:i + patch_size, j:j + patch_size] = graycoprops(patch_glcm, 'correlation')[0, 0]
            entropy_map[i:i + patch_size, j:j + patch_size] = -np.sum(patch_glcm * np.log2(patch_glcm + (patch_glcm == 0)))

    # Compute global averages of the feature maps
    features = {
        "GLCM_Contrast": np.mean(contrast_map),
        "GLCM_Homogeneity": np.mean(homogeneity_map),
        "GLCM_Energy": np.mean(energy_map),
        "GLCM_Correlation": np.mean(correlation_map),
        "GLCM_Entropy": np.mean(entropy_map),
        "GLCM_Mean": np.mean(resized_image),
        "GLCM_Std_Dev": np.std(resized_image),
        "GLCM_Variance": np.var(resized_image)
    }
    return features

# Initialize a list to store feature data for the DataFrame
feature_data = []

# Loop through each class folder
for class_name in os.listdir(dataset_folder):
    class_path = os.path.join(dataset_folder, class_name)
    
    # Process only directories
    if os.path.isdir(class_path):
        total_files = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg', '.jfif'))])
        
        with tqdm(total=total_files, desc=f"Processing {class_name} images", unit="file") as pbar:
            for file_name in os.listdir(class_path):
                if file_name.endswith(('.jpg', '.png', '.jpeg', '.jfif')):
                    image_path = os.path.join(class_path, file_name)
                    
                    # Preprocess the image
                    preprocessed_image = preprocess_image(image_path)

                    # Extract GLCM features
                    glcm_features = calculate_glcm_features(preprocessed_image)

                    # Perform DWT
                    coeffs_dwt = pywt.dwt2(preprocessed_image, 'haar')
                    LL_dwt, (LH_dwt, HL_dwt, HH_dwt) = coeffs_dwt
                    
                    # Extract DWT features
                    dwt_features = {
                        "DWT_LL": calculate_wavelet_features(LL_dwt),
                        "DWT_LH": calculate_wavelet_features(LH_dwt),
                        "DWT_HL": calculate_wavelet_features(HL_dwt),
                        "DWT_HH": calculate_wavelet_features(HH_dwt)
                    }

                    # Perform SWT
                    coeffs_swt = pywt.swt2(preprocessed_image, 'haar', level=1)
                    LL_swt, (LH_swt, HL_swt, HH_swt) = coeffs_swt[0]

                    # Extract SWT features
                    swt_features = {
                        "SWT_LL": calculate_wavelet_features(LL_swt),
                        "SWT_LH": calculate_wavelet_features(LH_swt),
                        "SWT_HL": calculate_wavelet_features(HL_swt),
                        "SWT_HH": calculate_wavelet_features(HH_swt)
                    }

                    # Combine all features
                    combined_features = {
                        "Class": class_name,
                        "Image Name": file_name,
                        **glcm_features,
                        **{f"DWT_{key}_{k}": v for key, subband in dwt_features.items() for k, v in subband.items()},
                        **{f"SWT_{key}_{k}": v for key, subband in swt_features.items() for k, v in subband.items()}
                    }

                    # Append features
                    feature_data.append(combined_features)
                    pbar.update(1)

# Create a DataFrame from the feature data
df = pd.DataFrame(feature_data)

# Save the DataFrame to a CSV file
output_csv_path = r"D:\new dataset\new dataset\SWT+DWT+GLCMcombined_features.csv"
df.to_csv(output_csv_path, index=False)

print("Feature extraction completed. Data saved to:", output_csv_path)


**CNN Features Extraction**

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.feature_selection import VarianceThreshold
from tqdm import tqdm

# Step 1: Load the Pre-trained CNN Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Step 2: Define Dataset Directory and Output File
dataset_dir =r"D:\new dataset\KTH_TIPS" # Replace with your dataset path
output_csv_path = r"D:\new dataset\new dataset\cnn\CNN_features.csv"  # Replace with desired CSV output path

# Step 3: Initialize Lists to Store Features, Labels, and Filenames
features_list = []
labels_list = []
image_filenames = []

# Step 4: Process Each Image in the Subfolders
for subdir, _, files in os.walk(dataset_dir):
    label = os.path.basename(subdir)  # Use subfolder name as the label
    if label == os.path.basename(dataset_dir):
        continue  # Skip the root folder

    print(f"Processing images in folder: {label}")
    for file in tqdm(files, desc=f"Processing {label}"):
        file_path = os.path.join(subdir, file)
        
        try:
            # Load and preprocess the image
            image = load_img(file_path, target_size=(224, 224))  # Resize to match VGG16 input
            image_array = img_to_array(image)
            image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
            image_array = preprocess_input(image_array)  # Preprocess as per VGG16 requirements

            # Extract features using the CNN model
            features = base_model.predict(image_array)
            flattened_features = features.flatten()  # Flatten the features into a 1D array

            # Append data to lists
            features_list.append(flattened_features)
            labels_list.append(label)
            image_filenames.append(file)
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

# Step 5: Perform Feature Selection (Filter out low-variance features)
print("Performing feature selection to retain important features...")
features_array = np.array(features_list)
selector = VarianceThreshold(threshold=0.01)  # Set a variance threshold (adjustable)
selected_features = selector.fit_transform(features_array)

# Step 6: Save the Filtered Features to a CSV File
print("Saving filtered features to CSV...")
filtered_features_df = pd.DataFrame(selected_features)
filtered_features_df['category'] = labels_list  # Add labels as a column
filtered_features_df['filename'] = image_filenames  # Add filenames as a column

filtered_features_df.to_csv(output_csv_path, index=False)
print(f"Filtered features saved successfully to: {output_csv_path}")
